In [6]:
import pyodbc
import pandas as pd
import boto3
from botocore.exceptions import ClientError
import logging


sql = """SELECT * FROM "Mp2-Reporting"."Customer List"."odin_Customer_list_extract"."unpivot passthrough" """

local_filename = "passthrough_pivot.csv"

s3_file = local_filename

bucketName = "prod-smt-data-cache"


logging.basicConfig(filename='new.log', filemode='w', level=logging.DEBUG,
                    format='%(asctime)s:%(levelname)s:%(message)s')



def conn_util():
    try:
        host = ''
        port = '31010'
        uid = ''
        pwsd = ''
        driver = '/Library/Dremio/ODBC/lib/libdrillodbc_sbu.dylib'
        #driver='/usr/local/lib/libdrillodbc_sbu.dylib'
        cnxn = pyodbc.connect("Driver={};ConnectionType=Direct;HOST={};PORT={};AuthenticationType=Plain;UID={};PWD={}".format(
        driver, host, port, uid, pwsd), autocommit=True)

        return cnxn

    except:
        logging.debug("check connection variable")


def pivotCaller():
    try:
        data = pd.read_sql(sql, conn_util())
        data['values'] = 1
        data = data.pivot(index='id', columns='Passthroughs', values='values')
        data = data.fillna(0)
        data.to_csv(local_filename, index=True)
        return logging.info("csv file extract was successful")

    except ValueError as v:

        logging.debug("Index contains duplicate entries, cannot reshape", v)

